<a href="https://colab.research.google.com/github/nathalia-acordi/warehouse-location/blob/main/Warehouse_Location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq pyomo
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.5 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [2]:
from pyomo.environ import *
import random

# Função para criar e resolver uma instância do problema
def solve_instance(instance_id):
    I = [0, 1, 2, 3]  # Exemplos de armazéns
    J = [0, 1, 2, 3, 4]  # Exemplos de clientes

    # Gerar parâmetros aleatórios para a instância
    f = [random.randint(800, 1200), random.randint(1200, 1800), random.randint(1500, 2500), random.randint(1000, 2000)]
    c = [[random.randint(1, 5) for _ in J] for _ in I]
    d = [random.randint(50, 300) for _ in J]
    C = [random.randint(400, 700) for _ in I]

    # Criar o modelo de otimização
    model = ConcreteModel()

    # Variáveis de decisão
    model.y = Var(I, within=Binary)
    model.x = Var(I, J, within=Binary)

    # Função objetivo
    model.obj = Objective(expr=sum(f[i] * model.y[i] for i in I) +
                             sum(c[i][j] * model.x[i, j] for i in I for j in J),
                          sense=minimize)

    # Restrições
    model.demand_constraint = ConstraintList()
    for j in J:
        model.demand_constraint.add(sum(model.x[i, j] for i in I) == 1)

    model.capacity_constraint = ConstraintList()
    for i in I:
        model.capacity_constraint.add(sum(d[j] * model.x[i, j] for j in J) <= C[i] * model.y[i])

    # Resolver o problema
    solver = SolverFactory('glpk')
    results = solver.solve(model)

    # Imprimir resultados
    print(f"\nInstância {instance_id}:")
    print(results)
    print("Status:", results.solver.status)
    print("Termination Condition:", results.solver.termination_condition)

    print("Custo Total =", model.obj())
    for i in I:
        print("Armazém", i, "está", "aberto" if model.y[i].value == 1 else "fechado")
        for j in J:
            if model.x[i, j].value == 1:
                print("  Cliente", j, "é atendido pelo armazém", i)

# Resolver 10 instâncias
for instance_id in range(1, 11):
    solve_instance(instance_id)



Instância 1:

Problem: 
- Name: unknown
  Lower bound: 2313.0
  Upper bound: 2313.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 24
  Number of nonzeros: 44
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 9
      Number of created subproblems: 9
  Error rc: 0
  Time: 0.007288217544555664
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Status: ok
Termination Condition: optimal
Custo Total = 2313.0
Armazém 0 está aberto
  Cliente 1 é atendido pelo armazém 0
  Cliente 4 é atendido pelo armazém 0
Armazém 1 está aberto
  Cliente 0 é atendido pelo armazém 1
  Cliente 2 é atendido pelo armazém 1
  Cliente 3 é atendido pelo armazém 1
Armazém 2 está fechado
Armazém 3 está fechado

Instância 2:

Problem: 
- Name: unknown
  Lower bound: 2725.0
  Upper bound: 2725.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 24
